# 导入库函数

In [1]:
from __future__ import division
from __future__ import print_function
from gravity_gae.evaluation import compute_scores
from gravity_gae.input_data import load_data
from gravity_gae.model import *
from gravity_gae.optimizer import OptimizerAE, OptimizerVAE
from gravity_gae.preprocessing import *
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错

# 定义超参数

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.logging.set_verbosity(tf.logging.ERROR)

flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('dataset', 'Cornell', 'Name of the graph dataset')

# Select machine learning task to perform on graph
flags.DEFINE_string('task', 'task_3', 'Name of the link prediction task')
''' See section 4.1. of paper for details about tasks:

- task_1: General Directed Link Prediction

- task_2: Biased Negative Samples Directed Link Prediction

- task_3: Bidirectionality Prediction
'''

# Model
flags.DEFINE_string('model', 'gravity_gcn_vae', 'Name of the model')
''' Available Models:

- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer
          GCN encoder and inner product decoder

- gcn_vae: Variational Graph Autoencoder from Kipf and Welling (2016), with
           Gaussian priors, 2-layer GCN encoders and inner product decoder

- source_target_gcn_ae: Source-Target Graph Autoencoder, as introduced
                        in section 2.6 of paper, with 2-layer GCN encoder
                        and asymmetric inner product decoder

- source_target_gcn_vae: Source-Target Graph Variational Autoencoder, as
                         introduced in section 2.6, with Gaussian priors,
                         2-layer GCN encoders and asymmetric inner product
 
- gravity_gcn_ae: Gravity-Inspired Graph Autoencoder, as introduced in
                  section 3.3 of paper, with 2-layer GCN encoder and 
                  gravity-inspired asymmetric decoder
 
- gravity_gcn_vae: Gravity-Inspired Graph Variational Autoencoder, as
                   introduced in section 3.4 of paper, with Gaussian 
                   priors, 2-layer GCN encoders and gravity-inspired decoder
'''

# Model parameters
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_integer('epochs', 200, 'Number of epochs in training.')
flags.DEFINE_boolean('features', False, 'Include node features or not in GCN')
flags.DEFINE_float('lamb', 1., 'lambda parameter from Gravity AE/VAE models \
                                as introduced in section 3.5 of paper, to \
                                balance mass and proximity terms')
flags.DEFINE_float('learning_rate', 0.1, 'Initial learning rate (with Adam)')
flags.DEFINE_integer('hidden', 24, 'Number of units in GCN hidden layer.')
flags.DEFINE_integer('dimension', 12, 'Dimension of GCN output: \
- equal to embedding dimension for standard AE/VAE and source-target AE/VAE \
- equal to (embedding dimension - 1) for gravity-inspired AE/VAE, as the \
last dimension captures the "mass" parameter tilde{m}')
flags.DEFINE_boolean('normalize', False, 'Whether to normalize embedding \
                                          vectors of gravity models')
flags.DEFINE_float('epsilon', 0.01, 'Add epsilon to distances computations \
                                       in gravity models, for numerical \
                                       stability')
# Experimental setup parameters
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 5., 'Proportion of edges in validation set \
                                   (for Task 1)')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      (for Tasks 1 and 2)')
flags.DEFINE_boolean('validation', False, 'Whether to report validation \
                                           results  at each epoch (for \
                                           Task 1)')
flags.DEFINE_boolean('verbose', True, 'Whether to print comments details.')

In [3]:
# Lists to collect average results
mean_roc = []
mean_ap = []
mean_time = []


# Load graph dataset
if FLAGS.verbose:
    print("Loading data...")
adj_init, features = load_data(FLAGS.dataset)


# The entire training process is repeated FLAGS.nb_run times
for i in range(FLAGS.nb_run):

    # Edge Masking: compute Train/Validation/Test set
    if FLAGS.verbose:
        print("Masking test edges...")

    if FLAGS.task == 'task_1':
        # Edge masking for General Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_general_link_prediction(adj_init, FLAGS.prop_test,
                                                FLAGS.prop_val)
    elif FLAGS.task == 'task_2':
        # Edge masking for B.N.S. Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_biased_negative_samples(adj_init, FLAGS.prop_test)
    elif FLAGS.task == 'task_3':
        # Edge masking for Bidirectionality Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_bidirectional_link_prediction(adj_init)
    else:
        raise ValueError('Undefined task!')

    # Preprocessing and initialization
    if FLAGS.verbose:
        print("Preprocessing and Initializing...")
    # Compute number of nodes
    num_nodes = adj.shape[0]
    # If features are not used, replace feature matrix by identity matrix
    if not FLAGS.features:
        features = sp.identity(adj.shape[0])
    # Preprocessing on node features
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape=())
    }

    # Create model
    model = None
    if FLAGS.model == 'gcn_ae':
        # Standard Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # Standard Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes,
                            features_nonzero)
    elif FLAGS.model == 'source_target_gcn_ae':
        # Source-Target Graph Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelAE(placeholders, num_features,
                                     features_nonzero)
    elif FLAGS.model == 'source_target_gcn_vae':
        # Source-Target Graph Variational Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelVAE(placeholders, num_features,
                                      num_nodes, features_nonzero)
    elif FLAGS.model == 'gravity_gcn_ae':
        # Gravity-Inspired Graph Autoencoder
        model = GravityGCNModelAE(placeholders, num_features,
                                  features_nonzero)
    elif FLAGS.model == 'gravity_gcn_vae':
        # Gravity-Inspired Graph Variational Autoencoder
        model = GravityGCNModelVAE(placeholders, num_features, num_nodes,
                                   features_nonzero)
    else:
        raise ValueError('Undefined model!')

    # Optimizer (see tkipf/gae original GAE repository for details)
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]
                                                - adj.sum()) * 2)
    with tf.name_scope('optimizer'):
        # Optimizer for Non-Variational Autoencoders
        if FLAGS.model in ('gcn_ae', 'source_target_gcn_ae', 'gravity_gcn_ae'):
            opt = OptimizerAE(preds = model.reconstructions,
                              labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices = False), [-1]),
                              pos_weight = pos_weight,
                              norm = norm)
        # Optimizer for Variational Autoencoders
        elif FLAGS.model in ('gcn_vae', 'source_target_gcn_vae', 'gravity_gcn_vae'):
            opt = OptimizerVAE(preds = model.reconstructions,
                               labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                             validate_indices = False), [-1]),
                               model = model,
                               num_nodes = num_nodes,
                               pos_weight = pos_weight,
                               norm = norm)

    # Normalization and preprocessing on adjacency matrix
    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    # Initialize TF session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Model training
    if FLAGS.verbose:
        print("Training...")
    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(FLAGS.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,
                                        placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        # Weight update
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],
                        feed_dict = feed_dict)
        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display epoch information
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                  "time=", "{:.5f}".format(time.time() - t))
            # Validation (implemented for Task 1 only)
            if FLAGS.validation and FLAGS.task == 'task_1':
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: FLAGS.dropout})
                val_roc, val_ap = compute_scores(val_edges, val_edges_false, emb)
                print("val_roc=", "{:.5f}".format(val_roc), "val_ap=", "{:.5f}".format(val_ap))

    # Compute total running time
    mean_time.append(time.time() - t_start)

    # Get embedding from model
    emb = sess.run(model.z_mean, feed_dict = feed_dict)

    # Test model
    if FLAGS.verbose:
        print("Testing model...")
    # Compute ROC and AP scores on test sets
    roc_score, ap_score = compute_scores(test_edges, test_edges_false, emb)
    # Append to list of scores over all runs
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# Report final results
print("\nTest results for", FLAGS.model,
      "model on", FLAGS.dataset, "on", FLAGS.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Loading data...
Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.57523 time= 0.12400
Epoch: 0002 train_loss= 1.35512 time= 0.00310
Epoch: 0003 train_loss= 1.14451 time= 0.00290
Epoch: 0004 train_loss= 0.80497 time= 0.00308
Epoch: 0005 train_loss= 0.77999 time= 0.00298
Epoch: 0006 train_loss= 0.79737 time= 0.00291
Epoch: 0007 train_loss= 0.71308 time= 0.00297
Epoch: 0008 train_loss= 0.62923 time= 0.00301
Epoch: 0009 train_loss= 0.61081 time= 0.00300
Epoch: 0010 train_loss= 0.63003 time= 0.00300
Epoch: 0011 train_loss= 0.64732 time= 0.00301
Epoch: 0012 train_loss= 0.64769 time= 0.00300
Epoch: 0013 train_loss= 0.62424 time= 0.00299
Epoch: 0014 train_loss= 0.59573 time= 0.00300
Epoch: 0015 train_loss= 0.56789 time= 0.00200
Epoch: 0016 train_loss= 0.55220 time= 0.00200
Epoch: 0017 train_loss= 0.54910 time= 0.00301
Epoch: 0018 train_loss= 0.54703 time= 0.00299
Epoch: 0019 train_loss= 0.54500 time= 0

Epoch: 0197 train_loss= 0.43030 time= 0.00301
Epoch: 0198 train_loss= 0.42684 time= 0.00301
Epoch: 0199 train_loss= 0.42412 time= 0.00301
Epoch: 0200 train_loss= 0.42822 time= 0.00299
Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.58152 time= 0.09306
Epoch: 0002 train_loss= 1.30586 time= 0.00331
Epoch: 0003 train_loss= 0.99652 time= 0.00306
Epoch: 0004 train_loss= 0.71997 time= 0.00298
Epoch: 0005 train_loss= 0.87104 time= 0.00298
Epoch: 0006 train_loss= 0.81017 time= 0.00304
Epoch: 0007 train_loss= 0.70376 time= 0.00302
Epoch: 0008 train_loss= 0.61613 time= 0.00293
Epoch: 0009 train_loss= 0.60406 time= 0.00301
Epoch: 0010 train_loss= 0.61843 time= 0.00299
Epoch: 0011 train_loss= 0.64492 time= 0.00300
Epoch: 0012 train_loss= 0.64029 time= 0.00301
Epoch: 0013 train_loss= 0.61810 time= 0.00299
Epoch: 0014 train_loss= 0.57462 time= 0.00300
Epoch: 0015 train_loss= 0.56518 time= 

Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.59607 time= 0.10042
Epoch: 0002 train_loss= 1.38605 time= 0.00300
Epoch: 0003 train_loss= 1.15893 time= 0.00399
Epoch: 0004 train_loss= 0.78985 time= 0.00300
Epoch: 0005 train_loss= 0.80228 time= 0.00301
Epoch: 0006 train_loss= 0.86779 time= 0.00299
Epoch: 0007 train_loss= 0.78103 time= 0.00299
Epoch: 0008 train_loss= 0.68302 time= 0.00301
Epoch: 0009 train_loss= 0.60818 time= 0.00300
Epoch: 0010 train_loss= 0.61371 time= 0.00200
Epoch: 0011 train_loss= 0.61472 time= 0.00300
Epoch: 0012 train_loss= 0.65605 time= 0.00301
Epoch: 0013 train_loss= 0.64178 time= 0.00301
Epoch: 0014 train_loss= 0.63041 time= 0.00299
Epoch: 0015 train_loss= 0.59382 time= 0.00299
Epoch: 0016 train_loss= 0.57775 time= 0.00301
Epoch: 0017 train_loss= 0.56032 time= 0.00300
Epoch: 0018 train_loss= 0.54705 time= 0.00301
Epoch: 0019 train_loss= 0.55233 time= 0.00299
Epoch: 00

Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.68699 time= 0.11588
Epoch: 0002 train_loss= 1.28636 time= 0.00418
Epoch: 0003 train_loss= 0.92472 time= 0.00434
Epoch: 0004 train_loss= 0.73324 time= 0.00406
Epoch: 0005 train_loss= 0.87455 time= 0.00495
Epoch: 0006 train_loss= 0.80025 time= 0.00500
Epoch: 0007 train_loss= 0.66475 time= 0.00300
Epoch: 0008 train_loss= 0.61587 time= 0.00299
Epoch: 0009 train_loss= 0.62519 time= 0.00400
Epoch: 0010 train_loss= 0.64517 time= 0.00511
Epoch: 0011 train_loss= 0.65781 time= 0.00289
Epoch: 0012 train_loss= 0.63177 time= 0.00299
Epoch: 0013 train_loss= 0.59374 time= 0.00408
Epoch: 0014 train_loss= 0.56874 time= 0.00395
Epoch: 0015 train_loss= 0.55329 time= 0.00309
Epoch: 0016 train_loss= 0.54819 time= 0.00394
Epoch: 0017 train_loss= 0.56310 time= 0.00399
Epoch: 0018 train_loss= 0.55547 time= 0.00400
Epoch: 0019 train_loss= 0.55606 time= 0.00400
Epoch: 00

Training...
Epoch: 0001 train_loss= 1.51760 time= 0.12460
Epoch: 0002 train_loss= 1.41377 time= 0.00300
Epoch: 0003 train_loss= 1.16447 time= 0.00400
Epoch: 0004 train_loss= 0.87641 time= 0.00318
Epoch: 0005 train_loss= 0.78936 time= 0.00305
Epoch: 0006 train_loss= 0.85485 time= 0.00399
Epoch: 0007 train_loss= 0.76617 time= 0.00299
Epoch: 0008 train_loss= 0.66507 time= 0.00308
Epoch: 0009 train_loss= 0.60615 time= 0.00299
Epoch: 0010 train_loss= 0.60866 time= 0.00310
Epoch: 0011 train_loss= 0.63247 time= 0.00397
Epoch: 0012 train_loss= 0.64178 time= 0.00293
Epoch: 0013 train_loss= 0.63672 time= 0.00454
Epoch: 0014 train_loss= 0.61438 time= 0.00251
Epoch: 0015 train_loss= 0.58048 time= 0.00295
Epoch: 0016 train_loss= 0.56864 time= 0.00295
Epoch: 0017 train_loss= 0.55019 time= 0.00411
Epoch: 0018 train_loss= 0.54895 time= 0.00294
Epoch: 0019 train_loss= 0.55216 time= 0.00304
Epoch: 0020 train_loss= 0.54323 time= 0.00402
Epoch: 0021 train_loss= 0.53919 time= 0.00293
Epoch: 0022 train_loss

...

Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.68340 time= 0.15049
Epoch: 0002 train_loss= 1.33188 time= 0.00311
Epoch: 0003 train_loss= 1.14847 time= 0.00309
Epoch: 0004 train_loss= 0.77610 time= 0.00304
Epoch: 0005 train_loss= 0.81364 time= 0.00503
Epoch: 0006 train_loss= 0.83417 time= 0.00399
Epoch: 0007 train_loss= 0.73340 time= 0.00394
Epoch: 0008 train_loss= 0.63932 time= 0.00403
Epoch: 0009 train_loss= 0.61766 time= 0.00402
Epoch: 0010 train_loss= 0.63840 time= 0.00403
Epoch: 0011 train_loss= 0.63934 time= 0.00393
Epoch: 0012 train_loss= 0.65290 time= 0.00297
Epoch: 0013 train_loss= 0.61936 time= 0.00303
Epoch: 0014 train_loss= 0.60279 time= 0.00297
Epoch: 0015 train_loss= 0.58342 time= 0.00299
Epoch: 0016 train_loss= 0.56514 time= 0.00300
Epoch: 0017 train_loss= 0.56220 time= 0.00405
Epoch: 0018 train_loss= 0.56119 time= 0.00292
Epoch: 0019 train_loss= 0.55972 time= 0.00348
Epoch: 00

Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.45425 time= 0.15260
Epoch: 0002 train_loss= 1.36122 time= 0.00493
Epoch: 0003 train_loss= 0.97948 time= 0.00300
Epoch: 0004 train_loss= 0.74741 time= 0.00401
Epoch: 0005 train_loss= 0.86445 time= 0.00300
Epoch: 0006 train_loss= 0.77712 time= 0.00499
Epoch: 0007 train_loss= 0.66990 time= 0.00309
Epoch: 0008 train_loss= 0.60203 time= 0.00305
Epoch: 0009 train_loss= 0.60736 time= 0.00294
Epoch: 0010 train_loss= 0.63511 time= 0.00311
Epoch: 0011 train_loss= 0.64567 time= 0.00400
Epoch: 0012 train_loss= 0.63282 time= 0.00395
Epoch: 0013 train_loss= 0.60502 time= 0.00301
Epoch: 0014 train_loss= 0.57293 time= 0.00305
Epoch: 0015 train_loss= 0.56489 time= 0.00295
Epoch: 0016 train_loss= 0.55496 time= 0.00201
Epoch: 0017 train_loss= 0.55608 time= 0.00305
Epoch: 0018 train_loss= 0.55931 time= 0.00306
Epoch: 0019 train_loss= 0.55168 time= 

Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.52390 time= 0.15359
Epoch: 0002 train_loss= 1.34586 time= 0.00404
Epoch: 0003 train_loss= 1.08682 time= 0.00300
Epoch: 0004 train_loss= 0.73849 time= 0.00397
Epoch: 0005 train_loss= 0.81256 time= 0.00412
Epoch: 0006 train_loss= 0.84920 time= 0.00394
Epoch: 0007 train_loss= 0.71805 time= 0.00398
Epoch: 0008 train_loss= 0.62160 time= 0.00306
Epoch: 0009 train_loss= 0.58757 time= 0.00507
Epoch: 0010 train_loss= 0.61896 time= 0.00393
Epoch: 0011 train_loss= 0.63421 time= 0.00399
Epoch: 0012 train_loss= 0.64191 time= 0.00402
Epoch: 0013 train_loss= 0.60870 time= 0.00411
Epoch: 0014 train_loss= 0.59842 time= 0.00423
Epoch: 0015 train_loss= 0.55563 time= 0.00332
Epoch: 0016 train_loss= 0.54256 time= 0.00399
Epoch: 0017 train_loss= 0.54011 time= 0.00300
Epoch: 0018 train_loss= 0.53700 time= 0.00397
Epoch: 0019 train_loss= 0.54112 time= 0.00303
Epoch: 00

Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.72104 time= 0.17272
Epoch: 0002 train_loss= 1.35808 time= 0.00301
Epoch: 0003 train_loss= 1.12387 time= 0.00296
Epoch: 0004 train_loss= 0.82885 time= 0.00306
Epoch: 0005 train_loss= 0.78368 time= 0.00398
Epoch: 0006 train_loss= 0.83873 time= 0.00401
Epoch: 0007 train_loss= 0.79116 time= 0.00399
Epoch: 0008 train_loss= 0.68101 time= 0.00401
Epoch: 0009 train_loss= 0.62577 time= 0.00403
Epoch: 0010 train_loss= 0.62439 time= 0.00397
Epoch: 0011 train_loss= 0.64143 time= 0.00296
Epoch: 0012 train_loss= 0.65044 time= 0.00316
Epoch: 0013 train_loss= 0.65496 time= 0.00288
Epoch: 0014 train_loss= 0.63092 time= 0.00299
Epoch: 0015 train_loss= 0.60504 time= 0.00303
Epoch: 0016 train_loss= 0.58470 time= 0.00294
Epoch: 0017 train_loss= 0.56901 time= 0.00301
Epoch: 0018 train_loss= 0.56408 time= 0.00337
Epoch: 0019 train_loss= 0.55813 time= 

Epoch: 0198 train_loss= 0.44113 time= 0.00304
Epoch: 0199 train_loss= 0.44232 time= 0.00306
Epoch: 0200 train_loss= 0.44267 time= 0.00197
Testing model...

Test results for gravity_gcn_vae model on Cornell on task_3 
 ___________________________________________________

AUC scores
 [0.8055555555555556, 0.7098765432098766, 0.7746913580246914, 0.8209876543209876, 0.8179012345679012, 0.7006172839506173, 0.6697530864197531, 0.691358024691358, 0.8055555555555555, 0.7006172839506173]
Mean AUC score:  0.7496913580246913 
Std of AUC scores:  0.057268682778024584 
 

AP scores 
 [0.8569210558457871, 0.7639839592872267, 0.8306933062680188, 0.8205111181212715, 0.8477821920081983, 0.759795227490661, 0.6410619278898375, 0.7332849309485931, 0.8446154939121693, 0.6839844276625887]
Mean AP score:  0.7782633639434351 
Std of AP scores:  0.07087810766730737 
 

Running times
 [0.7612240314483643, 0.7355003356933594, 0.7181615829467773, 0.8606541156768799, 0.8261828422546387, 0.8671460151672363, 0.895545